In [25]:
import glob
import os
import pandas as pd
import numpy as np
import shutil
from pathlib import Path

In [26]:
sddir = "/home/maik/b2drop/cosmicsense/inbox/fendt/timeseries/crns/JFC-1-sd"
remotedir = "/home/maik/b2drop/cosmicsense/inbox/fendt/timeseries/crns/JFC-1"
trgdir = "/media/x/cosmicsense/data/fendt/crns_v2"
tmpfile = "tmpfile.txt"
ids = [1, 2, 3, 4, 5, 6, 7, 8, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]

In [27]:
crns = {
     1: {"remotepattern": "up1_Data*.001*.txt",
         "sdpattern": "*.001",
         "replacetab": False,
         "colnames": ["rec_id", "datetime", "press1", "temp1", "relhum1", "volt", 
                      "counts1", "nsecs1", "counts2", "nsecs2"]},

     2: {"remotepattern": "up2_Data*.002*.txt",
         "sdpattern": "*.002",
         "replacetab": False,
         "colnames": ["rec_id", "datetime", "press1", "temp1", "relhum1", "volt",
                      "counts1", "nsecs1"]
        },

     3: {"remotepattern": "up3_Data*.003*.txt",
         "sdpattern": "*.003",
         "replacetab": False,
         "colnames": ["rec_id", "datetime", "press1", "temp1", "relhum1", "volt", 
                      "counts1", "nsecs1", "counts2", "nsecs2", "temp_ext"]
        },

     4: {"remotepattern": "up4_Data*.004*.txt",
         "sdpattern": "*.004",
         "replacetab": False,
         "colnames": ["rec_id", "datetime", "press1", "temp1", "relhum1", "volt",
                      "counts1", "nsecs1", "counts2", "nsecs2", "MetOne092_1",
                      "press4", "temp_ext", "relhum_ext","N1T_C","N1RH","N2T_C","N2RH"]
        },

     5: {"remotepattern": "kit5_Data.txt",
         "sdpattern": "*.005",
         "replacetab": False,
         # //RecordNum,Date Time(UTC),P4_mb,P1_mb,T1_C,RH1,T_CS215,RH_CS215,Vbat,N1Cts,N1ET_sec,N1T_C,N1RH
         "colnames": ["rec_id", "datetime", "press4", "press1", "temp1", "relhum1",
                      "temp_ext", "relhum_ext", "volt", "counts1", "nsecs1", "N1T_C", "N1RH"]
        },

     6: {"remotepattern": "kit6_Data.txt",
         "sdpattern": "*.006",
         "replacetab": False,
         # //RecordNum,Date Time(UTC),P4_mb,P1_mb,T1_C,RH1,T_CS215,RH_CS215,Vbat,N1Cts,N1ET_sec,N1T_C,N1RH
         "colnames": ["rec_id", "datetime", "press4", "press1", "temp1",
                      "relhum1", "temp_ext", "relhum_ext", "volt", "counts1", 
                      "nsecs1", "N1T_C", "N1RH"]
        },

     7: {"remotepattern": "kit7_Data.txt",
         "sdpattern": "NOT_AVAILABLE",
         "replacetab": False,
         # //RecordNum,Date Time(UTC),P4_mb,P1_mb,T1_C,RH1,T_CS215,RH_CS215,Vbat,N1Cts,N1ET_sec,N1T_C,N1RH
         "colnames": ["rec_id", "datetime", "press4", "press1", "temp1",
                      "relhum1", "temp_ext", "relhum_ext", "volt", "counts1", 
                      "nsecs1", "N1T_C", "N1RH"]
        },
    
     8: {"remotepattern": "kit8_Data.txt",
         "sdpattern": "NOT_AVAILABLE",
         "replacetab": False,
         # //RecordNum,Date Time(UTC),P4_mb,P1_mb,T1_C,T_CS215,RH1,RH_CS215,Vbat,N1Cts,N2Cts,N1ET_sec,N2ET_sec,N1T_C,N1RH,N2T_C,N2RH
         "colnames": ["rec_id", "datetime", "press4", "press1", "temp1", "temp_ext", 
                      "relhum1", "relhum_ext", "volt", "counts1", "counts2", "nsecs1", "nsecs2",
                      "N1T_C", "N1RH", "N2T_C", "N2RH"]
        },

    14: {"remotepattern": "CRSProbe_Data*.836*.txt",
         "sdpattern": "*.836",
         "replacetab": False,
         # //RecordNum,Date Time(UTC),P4_mb,P1_mb,T1_C,RH1,T_CS215,RH_CS215,Vbat,N1Cts,N1ET_sec,N1T_C,N1RH,
         "colnames": ["rec_id", "datetime", "press4", "press1", "temp1",
                      "relhum1", "temp_ext", "relhum_ext", "volt", "counts1",
                      "nsecs1", "N1T_C", "N1RH"]
        },

    15: {"remotepattern": "NOT_AVAILABLE",
         "sdpattern": "*.810",
         "replacetab": False,
         # //RecordNum,Date Time(UTC),P4_mb,P1_mb,T1_C,RH1,T_CS215,RH_CS215,Vbat,N1Cts,N1ET_sec,N1T_C,N1RH,
         "colnames": ["rec_id", "datetime", "press4", "press1", "temp1",
                      "relhum1", "temp_ext", "relhum_ext", "volt", "counts1",
                      "nsecs1", "N1T_C", "N1RH"]
        },

    16: {"remotepattern": "16_CRNS.txt",
         "sdpattern": "*.016",
         "replacetab": True,
         # //RecordNum,Date Time(UTC),P1_mb,T1_C,RH1,Vbat,N1Cts,N1ET_sec,N2Cts,N2ET_sec,T_CS215,RH_CS215,
         # rec_id	datetime	press1	temp1	relhum1	volt	counts1	nsecs1	counts2	nsecs2	temp_ext	relhum_ext
         "colnames": ["rec_id", "datetime", "press1", "temp1", "relhum1", "volt",
                      "counts1", "nsecs1", "counts2", "nsecs2", "temp_ext", "relhum_ext"]
        },

    17: {"remotepattern": "17_CRNS.txt",
         "sdpattern": "*.017",
         "replacetab": True,
         # //RecordNum,Date Time(UTC),P1_mb,T1_C,RH1,Vbat,N1Cts,N1ET_sec,N2Cts,N2ET_sec,T_CS215,RH_CS215,
         "colnames": ["rec_id", "datetime", "press1", "temp1", "relhum1", "volt", 
                      "counts1", "nsecs1", "counts2", "nsecs2", "temp_ext", "relhum_ext"]
        },
    
    18: {"remotepattern": "18_CRNS.txt",
         "sdpattern": "*.018",
         "replacetab": True,
         # //RecordNum,Date Time(UTC),P1_mb,T1_C,RH1,Vbat,N1Cts,N1ET_sec,N2Cts,N2ET_sec,T_CS215,RH_CS215,
         "colnames": ["rec_id", "datetime", "press1", "temp1", "relhum1", "volt", 
                      "counts1", "nsecs1", "counts2", "nsecs2", "temp_ext", "relhum_ext"]
        },

    19: {"remotepattern": "19_CRNS.txt",
         "sdpattern": "*.019",
         "replacetab": True,
         # //RecordNum,Date Time(UTC),P1_mb,T1_C,RH1,Vbat,N1Cts,N1ET_sec,T_CS215,RH_CS215,
         "colnames": ["rec_id", "datetime", "press1", "temp1", "relhum1", "volt",
                      "counts1", "nsecs1", "temp_ext", "relhum_ext"]
        },
    
    20: {"remotepattern": "NOT_AVAILABLE",
         "sdpattern": "*.cal",
         "replacetab": False,
         # //RecordNum,Date Time(UTC),Pv2.5_1,P1_mb,P4_mb,T1_C,RH1,Vbat,N1Cts,N1ET_sec,N2Cts,N2ET_sec,
         "colnames": ["rec_id", "datetime", "Pv2.5_1", "press1", "press4", "temp1", "relhum1", "volt", 
                      "counts1", "nsecs1", "counts2", "nsecs2"]
        },

    21: {"remotepattern": "CRSProbe_Data*.021*.txt",
         "sdpattern": "*.021",
         "replacetab": False,
         # //RecordNum,Date Time(UTC),P1_mb,P4_mb,T1_C,RH1,T_CS215,RH_CS215,Vbat,N1Cts,N1ET_sec,N1T_C,N1RH,
         "colnames": ["rec_id", "datetime", "press1", "press4", "temp1","relhum1",
                      "temp_ext", "relhum_ext", "volt", "counts1", "nsecs1", "N1T_C", "N1RH"]
        },

    22: {"remotepattern": "CRSProbe_Data*.022*.txt",
         "sdpattern": "*.022",
         "replacetab": False,
         # //RecordNum,Date Time(UTC),P1_mb,P4_mb,T1_C,RH1,T_CS215,RH_CS215,Vbat,N1Cts,N1ET_sec,N1T_C,N1RH,
         "colnames": ["rec_id", "datetime", "press1", "press4", "temp1","relhum1", 
                      "temp_ext", "relhum_ext", "volt", "counts1", "nsecs1", "N1T_C", "N1RH"]
        },

    23: {"remotepattern": "23_CRNS.txt",
         "sdpattern": "*.023",
         "replacetab": True,
         # //RecordNum,Date Time(UTC),P1_mb,T1_C,RH1,Vbat,N1Cts,N1ET_sec,N2Cts,N2ET_sec,T_CS215,RH_CS215,
         "colnames": ["rec_id", "datetime", "press1", "temp1", "relhum1", "volt",
                      "counts1", "nsecs1", "counts2", "nsecs2", "temp_ext", "relhum_ext"]
        },

    24: {"remotepattern": "24_CRNS.txt",
         "sdpattern": "*.024",
         "replacetab": True,
         # //RecordNum,Date Time(UTC),P1_mb,T1_C,RH1,Vbat,N1Cts,N1ET_sec,N2Cts,N2ET_sec,T_CS215,RH_CS215,
         "colnames": ["rec_id", "datetime", "press1", "temp1", "relhum1", "volt", 
                      "counts1", "nsecs1", "counts2", "nsecs2", "temp_ext", "relhum_ext"]
        },
    
    25: {"remotepattern": "kit25_Data.txt",
         "sdpattern": "NOT_AVAILABLE",
         "replacetab": False,
         # //RecordNum,Date Time(UTC),P4_mb,P1_mb,T1_C,RH1,T_CS215,RH_CS215,Vbat,N1Cts,N1ET_sec,N1T_C,N1RH
         "colnames": ["rec_id", "datetime", "press4", "press1", "temp1", "relhum1", 
                      "temp_ext", "relhum_ext", "volt", "counts1",  "nsecs1", "N1T_C", "N1RH"]
        }
}

In [28]:
for i, id in enumerate(ids):
    print("-------------")
    print("Processing %d" % id)

    try:
        os.remove(tmpfile)
    except:
        pass

    # REMOTE FILES
    print("Remote: ", end="")
    if crns[id]["remotepattern"]=="NOT_AVAILABLE":
        print("expected None...", end="")
    searchdir = os.path.join(remotedir,"%d" % id, crns[id]["remotepattern"])
    remotefiles = glob.glob(searchdir, recursive=True)
    print("found %d files" % len(remotefiles))

    for name in remotefiles:
        print(".", end="")
        fin = open(name, "r")
        body = fin.read()
        # replace comment character
        body = body.replace("//", "#")
        # replace zombie line endings
        body = body.replace(",\r\n", "\r\n")
        # comment out these lines
        body = body.replace("CRS#1:", "#CRS#1")
        body = body.replace("CRS#2:", "#CRS#2")
        if crns[id]["replacetab"]:
            body = body.replace("\t", ",")
        myfile = open(tmpfile, 'a')
        myfile.write(body)
        myfile.close()
    print("")

    # SD
    print("SD: ", end="")
    if crns[id]["sdpattern"]=="NOT_AVAILABLE":
        print("expected None...", end="")
    searchdir = os.path.join(sddir, "%d" % id)
    sdfiles = [filename for filename in Path(searchdir).glob("**/"+crns[id]["sdpattern"])]
    print("found %d files" % len(sdfiles))

    for name in sdfiles:
        print(".", end="")
        fin = open(name, "r")
        body = fin.read()
        # replace comment character
        body = body.replace("//", "#")
        # replace zombie line endings
        body = body.replace(",\r\n", "\r\n")
        body = body.replace(",\n", "\n")
        # comment out these lines
        body = body.replace("CRS#1:", "#CRS#1")
        body = body.replace("CRS#2:", "#CRS#2")
        myfile = open(tmpfile, 'a')
        myfile.write(body)
        myfile.close()
    print("")


    # MERGE
    df = pd.read_csv(tmpfile, sep=",", comment="#", header=None, error_bad_lines=False, warn_bad_lines=True)
    df.columns = crns[id]["colnames"]
    df.datetime = pd.to_datetime(df.datetime, infer_datetime_format=True)#format="%Y/%m/%d %H:%M:%S")
    df = df.set_index("datetime")
    df.insert(loc=1, column="datetime", value=df.index)
    df = df.sort_index()
    df = df[df.index >= "2019-07-25"]
    df = df[df.index <= "2019-05-01"]
    dupl = df.index.duplicated(keep='first')
    if np.any(dupl):
        print("Contains %d duplicates" % len(np.where(dupl)[0]))
        df = df[~dupl]
    fpath = os.path.join(trgdir, "%d_CRNS.txt" % id )
    df.to_csv(fpath, sep="\t", index=False, date_format="%Y-%m-%d %H:%M:%S")
    print("")

-------------
Processing 1
Remote: found 324 files
....................................................................................................................................................................................................................................................................................................................................
SD: found 5 files
.....

-------------
Processing 2
Remote: found 310 files
......................................................................................................................................................................................................................................................................................................................
SD: found 16 files
................

-------------
Processing 3
Remote: found 286 files
.....................................................................................................................................................

b'Skipping line 135: expected 8 fields, saw 15\nSkipping line 567: expected 8 fields, saw 15\nSkipping line 1739: expected 8 fields, saw 14\nSkipping line 3922: expected 8 fields, saw 15\n'


....................
SD: found 4 files
....

-------------
Processing 4
Remote: found 280 files
........................................................................................................................................................................................................................................................................................
SD: found 8 files
........

-------------
Processing 5
Remote: found 1 files
.
SD: found 14 files
..............

-------------
Processing 6
Remote: found 1 files
.
SD: found 10 files
..........

-------------
Processing 7
Remote: found 1 files
.
SD: expected None...found 0 files


-------------
Processing 8
Remote: found 1 files
.
SD: expected None...found 0 files


-------------
Processing 14
Remote: found 9 files
.........
SD: found 47 files
...............................................

-------------
Processing 15
Remote: expected None...found 0 files

SD: found 13 files
.............

-------------
Processing

ValueError: ('Unknown string format:', 'datetime')

In [29]:
df

,rec_id,datetime,press1,temp1,relhum1,volt,counts1,nsecs1,counts2,nsecs2,temp_ext,relhum_ext
0,rec_id,datetime,press1,temp1,relhum1,volt,counts1,nsecs1,counts2,nsecs2,temp_ext,relhum_ext
1,1,2019-05-14 13:45:00,953.4,11.9,62.9,13.277000000000001,27,50,21,51,NaN,NaN
2,2,2019-05-14 14:04:00,953.5,14.0,60.0,13.359000000000002,589,1140,352,1139,NaN,NaN
3,3,2019-05-14 14:24:00,953.0,13.0,57.1,13.697000000000001,654,1200,362,1200,NaN,NaN
4,4,2019-05-14 14:44:00,952.8,13.0,59.6,13.677,628,1200,412,1200,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
9158,2943,2019/07/17 11:51:00,944.1,28.2,57.0,13.563,679,1200,394,1200,24.7,48.9
9159,NPM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9160,NPM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9161,1,2019/06/06 16:24:22,944.7,16.4,66.4,12.747,0,0,0,0,-99.0,-99.0
